In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer

# Carregar os dados
file_path = 'temas_e_perguntas_limpo.csv'
data = pd.read_csv(file_path)

# Inicializar o modelo de embeddings
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Codificador de rótulos
le = LabelEncoder()

# Configurar validação cruzada estratificada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Métricas para avaliação
metrics = {"precision": [], "recall": [], "f1_score": [], "accuracy": []}

# Validação cruzada
for fold, (train_index, test_index) in enumerate(skf.split(data["Pergunta"], data["Tema"])):
    print(f"=== Fold {fold + 1} ===")
    
    # Dividir dados em treino e teste
    train_df = data.iloc[train_index]
    test_df = data.iloc[test_index]

    # Gerar embeddings
    train_embeddings = np.array([model.encode(text) for text in train_df["Pergunta"]])
    test_embeddings = np.array([model.encode(text) for text in test_df["Pergunta"]])

    # Codificar os rótulos (temas)
    y_train = le.fit_transform(train_df["Tema"])
    y_test = le.transform(test_df["Tema"])

    # Treinar o modelo: Rede Neural Simples
    classifier = MLPClassifier(hidden_layer_sizes=(128,), max_iter=500, random_state=42)
    classifier.fit(train_embeddings, y_train)

    # Previsões
    y_pred = classifier.predict(test_embeddings)

    # Avaliar as métricas
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)

    # Salvar as métricas do fold
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1_score"].append(f1)
    metrics["accuracy"].append(accuracy)

    # Exibir o relatório de classificação
    print("Relatório de Classificação para este fold:")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    print("\n")

# Resultados finais
print("=== Resultados Médios de Validação Cruzada ===")
for metric, scores in metrics.items():
    print(f"{metric.capitalize()}: {np.mean(scores):.2f}")


=== Fold 1 ===
Relatório de Classificação para este fold:
                                precision    recall  f1-score   support

                  Agropecuária       0.00      0.00      0.00         2
                   Agrotóxicos       0.50      0.67      0.57         3
                     Atmosfera       0.00      0.00      0.00         2
                         BRICS       0.50      0.33      0.40         3
                  BioGeografia       0.00      0.00      0.00         1
                      Biosfera       0.33      0.50      0.40         2
             Blocos Econômicos       0.00      0.00      0.00         2
    Capitalismo e Globalização       0.00      0.00      0.00         2
                   Cartografia       0.50      0.33      0.40         3
                  Centro-Oeste       0.33      0.50      0.40         2
                         China       0.50      0.33      0.40         3
                   Chuva Ácida       0.67      1.00      0.80         2
     

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import nltk

# Baixar stop words em português
nltk.download('stopwords')
portuguese_stop_words = stopwords.words('portuguese')

# Carregar os dados
file_path = 'temas_e_perguntas_limpo.csv'
data = pd.read_csv(file_path)

# Codificar os rótulos (Temas)
le = LabelEncoder()

# Configurar validação cruzada estratificada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Métricas para avaliação
metrics = {"precision": [], "recall": [], "f1_score": [], "accuracy": []}

# Inicializar o CountVectorizer para Bag of Words
vectorizer = CountVectorizer(stop_words=portuguese_stop_words)

# Validação cruzada
for fold, (train_index, test_index) in enumerate(skf.split(data["Pergunta"], data["Tema"])):
    print(f"=== Fold {fold + 1} ===")
    
    # Dividir dados em treino e teste
    train_df = data.iloc[train_index]
    test_df = data.iloc[test_index]

    # Transformar perguntas em uma matriz de contagem de palavras (Bag of Words)
    X_train = vectorizer.fit_transform(train_df["Pergunta"])
    X_test = vectorizer.transform(test_df["Pergunta"])

    # Codificar os rótulos (temas)
    y_train = le.fit_transform(train_df["Tema"])
    y_test = le.transform(test_df["Tema"])

    # Treinar o modelo: Random Forest
    classifier = RandomForestClassifier(random_state=42)
    classifier.fit(X_train, y_train)

    # Previsões
    y_pred = classifier.predict(X_test)

    # Avaliar as métricas
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)

    # Salvar as métricas do fold
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1_score"].append(f1)
    metrics["accuracy"].append(accuracy)

    # Exibir o relatório de classificação
    print("Relatório de Classificação para este fold:")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    print("\n")

# Resultados finais
print("=== Resultados Médios de Validação Cruzada ===")
for metric, scores in metrics.items():
    print(f"{metric.capitalize()}: {np.mean(scores):.2f}")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


=== Fold 1 ===
Relatório de Classificação para este fold:
                                precision    recall  f1-score   support

                  Agropecuária       0.00      0.00      0.00         2
                   Agrotóxicos       0.50      0.67      0.57         3
                     Atmosfera       0.50      0.50      0.50         2
                         BRICS       0.50      1.00      0.67         3
                  BioGeografia       0.00      0.00      0.00         1
                      Biosfera       1.00      0.50      0.67         2
             Blocos Econômicos       0.00      0.00      0.00         2
    Capitalismo e Globalização       1.00      0.50      0.67         2
                   Cartografia       1.00      0.67      0.80         3
                  Centro-Oeste       0.00      0.00      0.00         2
                         China       0.67      0.67      0.67         3
                   Chuva Ácida       1.00      1.00      1.00         2
     

In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Carregar os dados
file_path = 'temas_e_perguntas_limpo.csv'
data = pd.read_csv(file_path)

# Codificador de rótulos
le = LabelEncoder()

# Vetorizador Bag of Words
vectorizer = CountVectorizer()

# Configurar validação cruzada estratificada
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Métricas para avaliação
metrics = {"precision": [], "recall": [], "f1_score": [], "accuracy": []}

# Validação cruzada
for fold, (train_index, test_index) in enumerate(skf.split(data["Pergunta"], data["Tema"])):
    print(f"=== Fold {fold + 1} ===")
    
    # Dividir dados em treino e teste
    train_df = data.iloc[train_index]
    test_df = data.iloc[test_index]

    # Vetorização Bag of Words
    train_vectors = vectorizer.fit_transform(train_df["Pergunta"]).toarray()
    test_vectors = vectorizer.transform(test_df["Pergunta"]).toarray()

    # Codificar os rótulos (temas)
    y_train = le.fit_transform(train_df["Tema"])
    y_test = le.transform(test_df["Tema"])

    # Treinar o modelo: Rede Neural Simples
    classifier = MLPClassifier(hidden_layer_sizes=(128,), max_iter=500, random_state=42)
    classifier.fit(train_vectors, y_train)

    # Previsões
    y_pred = classifier.predict(test_vectors)

    # Avaliar as métricas
    precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)

    # Salvar as métricas do fold
    metrics["precision"].append(precision)
    metrics["recall"].append(recall)
    metrics["f1_score"].append(f1)
    metrics["accuracy"].append(accuracy)

    # Exibir o relatório de classificação
    print("Relatório de Classificação para este fold:")
    print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))
    print("\n")

# Resultados finais
print("=== Resultados Médios de Validação Cruzada ===")
for metric, scores in metrics.items():
    print(f"{metric.capitalize()}: {np.mean(scores):.2f}")


=== Fold 1 ===
Relatório de Classificação para este fold:
                                precision    recall  f1-score   support

                  Agropecuária       0.00      0.00      0.00         2
                   Agrotóxicos       0.33      0.67      0.44         3
                     Atmosfera       0.00      0.00      0.00         2
                         BRICS       1.00      1.00      1.00         3
                  BioGeografia       0.00      0.00      0.00         1
                      Biosfera       1.00      0.50      0.67         2
             Blocos Econômicos       0.20      0.50      0.29         2
    Capitalismo e Globalização       0.00      0.00      0.00         2
                   Cartografia       0.67      0.67      0.67         3
                  Centro-Oeste       1.00      0.50      0.67         2
                         China       1.00      0.67      0.80         3
                   Chuva Ácida       1.00      1.00      1.00         2
     